# Intro to git for Data Scientists

This is the first part of a 3-part `git` course for Data Scientists.

**Chapter I:**
* How to create a git repository (init, status)
* The four main stages of a file in a repository: untracked, tracked, staged and committed (ignore, add, commit, rm)

**Chapter II:**
* How to track changes of a file in a repository (diff, log)
* How to maintain different versions of a file without going crazy or using N suffixes (branch, checkout)
* Time-travel with git (revert, checkout, reset)

**Chapter III:**
* How to work with friends on the same file... and remain friends (merge)
* Working with remote repositories (clone, fetch, pull, push, pull requests)
* Contributing flows (forks, feature branches)

In this session we'll cover Chapter I!

## Chapter I

* How to create a git repository (init, status)
* The four main stages of a file in a repository: untracked, tracked, staged and committed (ignore, add, commit, rm)

**Warning**: This tutorial is meant to be executed ONCE and in the given order. Since in fact each step has side-effects, running one of the cells will likely change the output of re-running one of the previous cells. If you need to re-run previous cells, you will need to re-run all cells starting from the very first one below (cf. Creating a new repository)

## Creating a new repository
We will start by creating a git repository in a new subfolder of the folder where this notebook lives.

In [ ]:
%%bash 
rm -Rf my_first_repo
mkdir my_first_repo
git init my_first_repo

From now on, we have all the necessary components for git to start tracking changes in this repository. But what has the command exactly done? Let's check out the .git folder directory structure:

In [ ]:
%%bash 
ls -lai my_first_repo/.git

As you can see the `git init` command has created a bunch of files and directories inside the `.git` folder. These are the files that store all the information necessary to track and navigate through changes of the files in your repository. If you are curious (or even better when you understand the basics well), you can read more about each component's role in the [Pro git book (available for free online)](https://git-scm.com/book/en/v2/Git-Internals-Plumbing-and-Porcelain#ch10-git-internals).

We will now start introducing changes (that is, files) into our repository. Do we have anything in the repository yet? 

In [ ]:
%%bash 
cd my_first_repo
git status

The short answer is no, there's nothing in the repository because `git` only does what we tell it to do, and we haven't asked it to track any of the contents of the folder. That is why it lists the folder's contents as "untracked" files.

As a sidenote, git is telling us that we are working in our "master" branch, which is the default name for the default branch of any project. We'll talk about branches later but for now we can already notice that the status command is telling us that there are a few files that are "untracked".

## The first 2 out of 4 stages of a file in a repository

As we saw in the previous section, when we create a new repository, git will start noticing files that exist in the same folder but are not yet tracked by
git. This suggests that there are at least two stages in the lifetime a file living in the same folder of a git repository:
* untracked
* tracked

For example, suppose that we start working on a new file `test.py` containig code we want to start tracking under git:

In [ ]:
%%bash
echo "print('Complicated Python pipeline')" > my_first_repo/test.py

This new file (`test.py`) exists now in the repository but is not yet being tracked by git:

In [ ]:
%%bash
cd my_first_repo
git status

Git tells us that our new file `test.py` is **untracked**.

If we want to start tracking this new file within our repository, 
we just need to `add` it for the first time:

In [ ]:
%%bash
cd my_first_repo
git add test.py
git status

The `test.py` file we just created is now tracked by the repository, and git status tells us as much. 

Note that git will complain if we ask it to to track files that are in fact in folders other than the on where the repository was created:

In [ ]:
%%bash
cd my_first_repo
touch ../another_test.txt
# git add ../another_test.txt
# This will fail with an error:
# `fatal: ../another_test.txt: '../another_test.txt' is outside repository`

### IMPORTANT: Things git should not track...

There are some files that will be in the same folder as the repo but that we will never want to track. Examples of these are:
* data files,
* artifact files (serialized models, binaries),
* **credentials**,
* generally anything that is not source code.

Suppose that we have a `data.csv` file that we don't want to include in the repository:

In [ ]:
%%bash
touch my_first_repo/data.csv

This file will now show as an untracked file everytime we run `git status` which eventually will get a bit tiresome:

In [ ]:
%%bash
cd my_first_repo
git status

Since we don't want to track it, what we will do is tell `git` to **ignore** it. As a matter of fact, we will tell `git` to ignore all files ending with the
`.csv` extension. To do that, we just need to create a file named `.gitignore` in the repo, containing the line `*.csv`:

In [ ]:
%%bash
cd my_first_repo
echo "*.csv" > .gitignore
git add .gitignore
git status

Note that we also started tracking the `.gitignore` file. Magically, git status now doesn't show `data.csv` anymore: `git` knows files like that one can be safely ignored.

**Excercise 0.0**: Why do you think we need to track (`git add`) the `.gitignore` file?

**Excercise 0.1**: Can you think of (at least) two reasons why we typically don't want to track binary files in a git repository?

**Exercise 0.2**: How would we ignore a whole directory? And if we want to ignore a whole directory but one of its subfolders?

## The last two of the four stages of a file in a repository

We have seen that there are at least two stages in the lifetime a file living in the same folder of a git repository:
* untracked: when a file resides in the same folder where we created the repository, but it is not under source control
* tracked: when add a previously _untracked_ file into the repository.

There are in fact two additional stages for files _tracked_ by a repository:
* staged
* commited

As you saw, we started tracking a file `test.py` by git-adding it into the repository. As a matter of fact, the `git add` command
did actually two things:
* it told git to start tracking the contents of that file
* it _staged_ the changes in the file (i.e. its creation) to prepare them to be _commited_

You can think of _staging_ a file as the last step prior to making the file (or the changes introduced to the file) permanent in the repository. While a file is _staged_, you can continue to modify it:

In [ ]:
%%bash
cd my_first_repo
# we add one more line to our `test.py` script
echo "print('Now it is even more complicated!')" >> test.py
git status

And the file remains in its _staged_ state, but now `git status`
informs us that our latest changes have not yet been staged.

If we were to commit the _staged_ `test.py` changes now, we would not be "saving" the changes 
we just performed, because those changes haven't yet been staged. So let's stage them 
so that we can commit everything in one batch!

In [ ]:
%%bash
cd my_first_repo
git add test.py
git status

### Commiting the staged changes

In order to make the new files (or the changes in existing files) permanently stored in git, we need to _commit_ them.

Here we need to explain what we mean by "permantently storing". Permanently storing changes in git (or most version control systems) doesn't mean that we won't be able to (or that we shouldn't) change those files ever again. On the contrary, it means that we will store the state of the file in the repository in a way that will allow us to build on top of that state, coming back to it at any point in the future. You read well, *any* point in the future! Git is like a time-travel machine!

So let's _commit_ both of the staged changes (the ones in `.gitignore` and the ones in `test.py`):

In [ ]:
%%bash
cd my_first_repo
git commit -m 'My first commit and my first commit message!'

Now when we check the status of our repository, we notice that we no longer have staged
changes:

In [ ]:
%%bash
cd my_first_repo
git status

### The opposite of `add`ing is `rm`oving!

Sometimes we would like to stop tracking files we previously added to a repository, perhaps because they are no longer useful or because we added them mistakenly. 

There's a useful git command to do just that, which is `git rm`. To illustrate its behavior, we will first create, stage and commit a dummy file that we would later like to remove:

In [ ]:
%%bash
cd my_first_repo
touch dummy.py
git add dummy.py
git commit -m 'commiting a dummy file just to remove it later!'

Let's remove this dummy! 

In [ ]:
%%bash
cd my_first_repo
git rm dummy.py
git status

Suprise! Removing is dual to `add` in one more way: Applying `git rm` only _stages_ the delete operation but doesn't yet make it permanent. We need to further _commit_ it for the deletion to take place:

In [ ]:
%%bash
cd my_first_repo
git commit -m 'Away with the dummy!'
git status

**Exercise 0.3**: Let's practice a bit more with the concepts of tracking, staging and committing. To this end, please try to follow these steps in order:
1. Create two empty new files, start tracking and commit them (`A.txt` and `B.txt`). 
2. Change a few lines in `A.txt` and `B.txt` and stage the changes.
3. *Unstage* `B.txt`. Hint: Google `git reset HEAD`. 
4. Now commit the changes in `A.txt`
5. Stage `B.txt` and commit it (in a different commit!)

### [Advanced] Committing to the cause

As a `git` user, you will be constantly commiting work into your repository, so it may be interesting for you to learn more about the act of committing. This is an "advanced" section so feel free to skip it if it doesn't pick your interest!

#### How are commits identified?

Commits are identified by their commit `sha`. This is an alphanumerical key generated by a hashing or digesting function which takes a bunch of strings as its input and returns a probably-unique sha as its output. I say probably-unique because the truth is, there is a 1 in XXXXX change that your commit `sha` will be the same as someone else's in the universe. 

The elements that determine the value of a commit's sha are:
- author information
- committer information (not necessarily the same as the author!)
- sha's of parent commits
- sha of the "whole repo" (root tree)

references: 
* https://gist.github.com/masak/2415865
* https://blog.thoughtram.io/git/2014/11/18/the-anatomy-of-a-git-commit.html

#### What to put in a commit message?

If you google "how to write a good commit message" you will find hundreds of opinionated results giving advice on how
to perfectly craft a commit message.

While I have tried in the past to adopt some of the good advice, I find that in practice spending too much time writing a commit message makes the process inconsistent: one starts with very good intentions but soon abandons it. My simplified recommendations would be:
- Keep them short but descriptive
- State your intention, not what you did

#### How often do we commit?

In practice it is quite favorable to divide your work into "atomic" commits that can't be divided into smaller changes. This 
will make code review simpler for your collaborators and will make your change easier to roll-back in case of need. On the flip side, it means you will be committing quite often, which generates a bit of overhead.

#### Ammending

How many times have you wanted to _save_ your current work and typed:

> git commit -m 'WIP'

(WIP stands for work-in-progress)

If you haven't, congratulations! But if you, like the rest of us, have abused of this practice, know that 
there is a simple tool to support this workflow which is `git ammend`-ing. The `ammend` command allows you to literally ammend your previous commit by adding additional changes or changing its commit message. An even more powerful tool is `git rebase` which we will see later.

**Warning**: `git ammend` and `git rebase` are "history-changing" operations. That is, when you ammend a previous commit, its sha changes so any parents pointing to it become "orphan" (point to a commit that no longer exists!). This is a **dangerous** operation that you should perform only before pushing your changes to the remote repository - otherwise your _local_ and _remote_ repository will _diverge_. 


**Exercise 0.4**: In this exercise we will practice the use of `ammend` and will experiment with what happens when two branches diverge.

## Take aways
And now we're truly done with this first part where we have learned:

0. The difference between _untracked_ and _tracked_ files in our repository
1. How to start tracking a file into a repository, or _stage_ changes we have done to an already tracked file
2. How to commit the _staged_ changes to make them permanent
3. How to _remove_ a previously tracked file from the respository